In [ ]:
FOLDER = 'C:\\Users\\Prakrith\\Desktop\\'

# 1. Project Modules

In [ ]:
import os, re, sys

In [ ]:
MK_PATH = os.path.split(os.getcwd())[0] + os.sep + 'mk'
sys.path.append(MK_PATH)

In [ ]:
from util import tools
from util.folder import Folder, Stack, Ilastik
from util.il import RunIL
from util.cp import RunCP
from util.results import ResultsImage, ResultsObject, heatmap

# 2. User Variables

In [ ]:
RE_TIF = re.compile("(.*)\.tif")
RE_PNG = re.compile("(.*)\.png")

In [ ]:
PRBSTG_DICT = {0:'_background_prbstg2_0.png', 1:'_cell_boundary_prbstg2_1.png', 2:'_mk_prbstg2_2.png', 3:'_pplt_prbstg2_3.png'}

In [ ]:
PIPELINE = tools.read_config('mk')
RULES = tools.read_config('rules')

# 3. Initialize

In [ ]:
stack = Stack(FOLDER, RE_TIF)
stack.df

In [ ]:
time = stack.slices

# 4. Setup 

In [ ]:
single = Folder(stack.folder('single_images'), RE_TIF)
single.df

In [ ]:
stack.count
stack.check(single.count)
stack.done

In [ ]:
stack.unpack(single.path, 8)
single.df

# 5. ilastik

## a) Setup

In [ ]:
ilastik = Ilastik(stack.folder('ilastik'), RE_PNG)
ilastik.df

In [ ]:
ilastik.count
ilastik.check(PRBSTG_DICT, single.df)
ilastik.incomplete

## b) Processing

In [ ]:
RunIL(ilastik.incomplete)
ilastik.df

# 6. Cellprofiler

## MK

In [ ]:
out = stack.folder('output')

In [ ]:
RunCP(PIPELINE, stack.path, 'mk.txt', single.list + ilastik.list, out, rules=RULES)

# 7. Results

## results_Image.csv

In [ ]:
res = ResultsImage(out, 'results_Image.csv', time)

In [ ]:
res.df

In [ ]:
x = res.analyze()

## results_mk_filter.csv

In [ ]:
res2 = ResultsObject(out, 'results_mk_filter.csv', time, 'MK')

In [ ]:
res2.df

In [ ]:
y = res2.analyze()

## results_pplt_filter.csv

In [ ]:
res3 = ResultsObject(out, 'results_pplt_filter.csv', time, 'PPLT')

In [ ]:
res3.df

In [ ]:
z = res3.analyze()

In [ ]:
heatmap(x,y,z,out)

In [ ]:
tools.move_results(out)